# Day 13: Shuttle Search

https://adventofcode.com/2020/day/13

## Part 1

In [473]:
def readInput13(filename):
    with open(filename) as f:
        lines = [ l.strip("\n") for l in f.readlines()]
    timestamp = int(lines[0])
    buses = []
    for e in enumerate (lines[1].split(",")):
        if e[1]=="x":
            continue
        buses.append((int(e[1]),e[0]))
    return timestamp, buses

timestamp, buses = readInput13("data/day13test0.txt")
timestamp, buses = readInput13("data/input13.txt")
timestamp, buses

(1000299,
 [(41, 0),
  (37, 35),
  (971, 41),
  (17, 58),
  (13, 59),
  (23, 64),
  (29, 70),
  (487, 72),
  (19, 91)])

In [475]:
nextdep = []

for b,d in buses:
    n = (int(timestamp/b)+1)*b
    nextdep.append(n)

leave = min(nextdep)
bus = activebuses[nextdep.index(leave)]
sol1 = bus*(leave-timestamp)

print("Solution Part 1 =",sol1)

Solution Part 1 = 156


## Part 2

Solution has to be multiple of first bus number (0 rest in division). Solution + the other dtime values must be multiple of all other bus numbers. Of course bus numbers are primes, so brute force will not work...

In [483]:
def bruteForcePart2(buses):
    n = buses[0][0]
    while(True):
        n += buses[0][0]
        match = 0
        for i in range(1,len(buses)):
            q = (n+buses[i][1]) % buses[i][0] # 0 if match condition for current bus
            if q!=0:
                break
            else:
                match += 1
        if match == len(buses)-1:
            return n
            break

In [486]:
# Brute force attempt on test, should return 1068781

timestamp, buses = readInput13("data/day13test0.txt")
n = bruteForcePart2(buses)
print("Solution test Part 2 =",n)

Solution test Part 2 = 1068781


Given the extremes of the problem, this has certainly to do with modular algebra, which I know very little about (I discever its extistence during last year AOC!). Following an hint from @zar I look up the [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem):

> If one knows the remainders of the Euclidean division of an integer n by several integers, then one can determine uniquely the remainder of the division of n by the product of these integers, under the condition that the divisors are pairwise coprime.

Ok! If I manage to implement Chinese Reminder Algorithm and apply it to the bus number and the *difference between the bus number and their delta time* (that would be the reminder of the division by the bus number of the result!) I'm done!

### Implementing Chinese Reminder Algorithm using Extended Euclid algorithm

Using my mplementaion of `EuclidGDC()`, `ExtendedEuclidGDC()` and `invM()` from my solution to Day 22 of AOC 2019

In [580]:
# Implementing Chinese Reminder Algorithm using Extended Euclid algorithm

def EuclidGDC(a,n):
    '''Euclid algorithm to computed GCD'''
    if n==0:
        return a
    else:
        return EuclidGDC(n, a%n)

def ExtendedEuclidGDC(a,n): 
    '''Extended Euclid algorithm to computed GCD and integer pair (x,y) that satifies d = gcd(a,n) = a*x+n*y'''
    if n==0:
        return a,1,0
    di, xi, yi = ExtendedEuclidGDC(n,a%n)
    d = di
    x = yi
    y = xi - a//n * yi # use // for integer division!
    return d, x, y 

def invM(a,m):
    '''Compute inverse of a mod m using Extended Euclid algorithm'''
    d,x,y = ExtendedEuclidGDC(a,m)
    return x

def minX_CTR(num, rem): 
    '''
    Return minimum integer satisfying Chinese Theoreme of Reminders equations.
    Assumes num and rem have the same size, and num components are coprimes.
    '''
    prod = 1
    for n in num: 
        prod = prod * n 
    result = 0
    for n,a in zip(num,rem): 
        pp = prod // n
        result = result + a*invM(pp,n)*pp 
    return result % prod 

In [581]:
num = [3, 4, 5] 
rem = [2, 3, 1] 
print(minX_CTR(num, rem)) 

11


In [582]:
testfiles = [
    "data/day13test0.txt", # 1068781
    "data/day13test1.txt", # 3417
    "data/day13test2.txt", # 754018
    "data/day13test3.txt", # 779210
    "data/day13test4.txt", # 1261476
    "data/day13test5.txt", # 1202161486
]

for f in testfiles:
    timestamp, buses = readInput13(f)
    n = []
    a = []
    for b,d in buses:
        n.append(b)
        a.append(b-d) # reminders are bus number minus delta time!
    sol = minX_CTR(n, a)
    print(n,a,sol)

[7, 13, 59, 31, 19] [7, 12, 55, 25, 12] 1068781
[17, 13, 19] [17, 11, 16] 3417
[67, 7, 59, 61] [67, 6, 57, 58] 754018
[67, 7, 59, 61] [67, 5, 56, 57] 779210
[67, 7, 59, 61] [67, 6, 56, 57] 1261476
[1789, 37, 47, 1889] [1789, 36, 45, 1886] 1202161486


In [583]:
timestamp, buses = readInput13("data/input13.txt")
n = []
a = []

for b,d in buses:
    n.append(b)
    a.append(b-d) # reminders are bus number minus delta time!

sol2 = minX_CTR(n, a)
print("Solution test Part 2 =",sol2)

Solution test Part 2 = 404517869995362
